In [1]:
import numpy as np
import pandas as pd
import tkinter as tk

In [2]:
#2081 rows 
news = pd.read_excel("hw1_text.xlsx", sheet_name="foxconn")
nrow = news.shape[0]
ncol = news.shape[1]
keyword = pd.read_excel(r'D:\Course\大數據與商業分析\HW2\bda2019_hw2_table.xlsx')
raw_query = pd.read_excel(r'D:\Course\大數據與商業分析\HW2\bda2019_hw2_table.xlsx', sheet_name='L2_query')

In [22]:
# 245 keywords
keyword_list = keyword["term"]

# 2081 articles
data = list(news['標題'] + news['內容'])

# 6 queries
query = list(raw_query['標題'] + raw_query['內容'])

In [23]:
# calculate tf and df of 245 keywords in foxconn
k = keyword_list[0]
key_tf = dict()
key_df = dict()
for i in range(len(keyword_list)):
    key_tf[keyword_list[i]] = 0
    key_df[keyword_list[i]] = 0
    for j in range(len(news["內容"])):
        if news["內容"][j].count(keyword_list[i])>0:
            key_tf[keyword_list[i]] += news["內容"][j].count(keyword_list[i])
            key_df[keyword_list[i]] += 1

In [24]:
# dict to list
key_list = []
tf_list = []
df_list = []
for key, value in key_tf.items():
    key_list.append(key)
    tf_list.append(value)
for key, value in key_df.items():
    df_list.append(value)

In [25]:
# get tf, df and tf-idf dataframe of keyword
c = {'keyword':key_list, 'tf':tf_list, 'df':df_list}
df = pd.DataFrame(c)
df['tf-idf'] = (1+np.log10(df['tf'])) * (np.log10(nrow/df['df']))
df.insert(0, 'number', np.arange(1,246))

In [26]:
# get 245 keywords tf in 6 queries 
query_tf = []
for q in query:
    tmp = []
    for k in keyword_list:
        tmp.append(q.count(k))
    query_tf.append(tmp)

# get 245 keywords tf in 2081 foxconn news
data_tf = []    
data_df = list(df['df'])
for d in data:
    tmp = []
    for k in keyword_list:
        tmp.append(d.count(k))            
    data_tf.append(tmp)


In [27]:
# We use tf-idf as vector to calculate the similarity between news.
# 1.Calculate tf-idf for data(2081 x 245) and query(6 x 245)
data_tfidf = []
for tf in data_tf: 
    tmp = []
    for i in range(len(keyword_list)):
        if tf[i] == 0 :
            tmp.append(0)
        else:
            tmp.append((1+np.log10(tf[i])) * (np.log10(len(data) / data_df[i] )))
    data_tfidf.append(tmp)
query_tfidf = []
for tf in query_tf:
    tmp = []
    for i in range(len(keyword_list)):
        if tf[i] == 0 :
            tmp.append(0)
        else:
            tmp.append((1+np.log10(tf[i])) * (np.log10(len(data) / data_df[i] )))
    query_tfidf.append(tmp)


In [28]:
# 2.Vector normalization(make vector length = 1)
query_vec = []
for tfidf in query_tfidf:
    sum = 0
    tmp = []
    for j in range(len(tfidf)):
        sum += tfidf[j] ** 2
    sum = np.sqrt(sum)
    for j in range(len(tfidf)):
        tmp.append(tfidf[j] / sum)
    query_vec.append(tmp)
     
data_vec = []
for tfidf in data_tfidf:
    sum = 0
    tmp = []
    for j in range(len(tfidf)):
        sum += tfidf[j] ** 2
    sum = np.sqrt(sum)
    for j in range(len(tfidf)):
        tmp.append(tfidf[j] / sum)
    data_vec.append(tmp)

In [29]:
# 3.Calculate similarity of 6 queries
sim = []
for q in range(len(query_vec)):
    tmp = []
    for d in range(len(data_vec)):
        cos = np.inner(query_vec[q],data_vec[d])
        tmp.append((d, cos))
    sim.append(tmp)


In [32]:
# Sort
for i in range(len(sim)):
    sim[i].sort(key=lambda x: x[1], reverse=True)

# Get top 3 
for d in sim:
    print("搜尋新聞")
    print(news.iloc[d[0][0]])
    for i in range(3):
        print("相關新聞" + str(i+1))
        print(news.iloc[d[i+1][0]]['標題'])
        print(news.iloc[d[i+1][0]]['內容'])
        print()
    print()
    print()
    print()

搜尋新聞
編號                                                   65
類別                                                 鴻海新聞
時間                                  2016-01-15 08:30:20
標題                                鴻海傳加碼至7,000億日圓搶親 夏普飆漲
內容    日本讀賣新聞報導，鴻海考慮提高投資夏普的金額，一口氣從5,000億日圓加碼至7,000億日圓...
Name: 64, dtype: object
相關新聞1
夏普否認給鴻海優先談判權
彭博資訊報導，日本夏普公司5日表示，否認給予鴻海優先交涉權的報導，指出鴻海作為優先合作夥伴的說法並非夏普的正式聲明，雙方達成的協議只是把鴻海收購提案的最後有效期限延長至2月29日。

相關新聞2
鴻夏若戀成 研調：日本面板受衝擊
針對鴻夏戀後續，研調機構WitsView指出，鴻海若與夏普結合，將對日本另一家小尺寸面板廠－日本顯示器公司（JapanDisplay;JDI）造成衝擊。WitsView直言，日本面板廠重視技術優勢，辛苦的維持產業地位，夏普若真下嫁鴻海，意味日本技術獨立性，以及JDI和夏普原先存在的平衡性將被打破。因此，鴻海在取得技術與產能後，勢必將透過系統整合，在市場上樹立新的業務模式。WitsView認為，一旦鴻海主導夏普經營，交易模式翻轉，屆時包括JDI等僅專注在面板業務的廠商，可能因為缺乏整合的談判籌碼，被迫選擇與垂直鏈廠商合作，在產業的「發言權」與影響力將逐漸被削弱。

相關新聞3
WSJ：鴻海需更多時間評估夏普債務資訊
華爾街日報報導，鴻海周三收到一份清單，上面列出100項夏普的「或有負債」（contingent liabilities），總金額約3,500億日圓，鴻海需更多時間評估新資訊，以降低未來的財務風險，才能繼續和夏普進行簽約事宜。報導指出，鴻海是24日收到夏普提供的新細節，表示須全盤了解狀況後才能繼續談判，不過還是希望很快能和夏普順利達成協議。夏普對此不願置評。




搜尋新聞
編號                                                  220
類

In [13]:
# Bonus #
# input the number of news you'd like to search
num = int(input("請輸入欲搜尋相關新聞編號(1~2081)"))

請輸入欲搜尋相關新聞編號22


In [40]:
# Calculate similarity between data and input 
sim = []
for d in range(len(data_vec)):
    cos = np.inner(data_vec[num-1],data_vec[d])
    sim.append((d, cos))
sim.sort(key=lambda x: x[1], reverse=True)
print("搜尋新聞")
print(news.iloc[sim[num-1][0]])
print()
# Show top 3 related news
for i in range(3):
    print("相關新聞" + str(i+1) + '      時間:' + str(news.iloc[sim[i+1][0]]['時間']))
    print(news.iloc[sim[i+1][0]]['標題'])
    print(news.iloc[sim[i+1][0]]['內容'])
    print()

搜尋新聞
編號                                                 1429
類別                                                 鴻海新聞
時間                                  2016-05-03 09:56:18
標題                                   報稅、信心面不足 台股跌破8300點
內容    台股今日為5月的第1個交易日，雖然美股止跌反彈百點，歐股也由汽車製造股領漲，不過，台股今日卻...
Name: 1428, dtype: object

相關新聞1      時間:2016-01-07 14:52:15
認售爆量 台股靜待回穩
台股今（7）日重挫、下殺取量，成交金額高達1,167億元，亞洲國家受陸股再度觸發熔斷機制影響，全面下挫，台灣加權指數終場大跌逾1.7%。權證今日多空比重呈五五波，認售權證完全獲得投資人青睞，認購權證比重甚至出現極其罕見地低於認售權證狀況，台股空頭氣焰強。權證隨大盤出量，早盤成交金額就來到17億元、全日成交超過28億元，金額一路從去年12月31日的12億元攀升向上。法人表示，國際資金近期受到沙伊二國衝突、人民幣急貶、甚至北韓宣稱試爆氫彈等黑天鵝侵擾，於全球股市展開大賣股。根據群益金鼎證券統計，今日盤中權證市場成交金額前十大權證，有九檔是認售權證，成交金額前三大分別是元大JQ（07856P）、HF凱基（08110P）、元大YU（08129P），連結標的分別是台積電、鴻海與TPK宸鴻。唯一一檔進榜的認購權證是連結可成的41元大（041462）。其餘進榜認售權證的連結標的相對集中，包括大立光、漢微科與聯發科。股王、股后同時成為認售權證爆量標的，法人指出，台股後市要振作，股王與股后重要性相當高，高價指標若能回穩，有助盤勢止穩。

相關新聞2      時間:2016-01-26 12:17:47
鴻海可成 權證交投熱
台股今（26）日雖受美股走跌、亞股欲振乏力的拖累中，早盤一失回測7,800點，但在市場追殺力道也有限中，配合國安等政府基金進場護盤，盤勢止穩；權證市場方面，截至午盤為止，市場交投轉趨活絡，成交金額最大的權證為群益GH（042627），連結標的為鴻海（2317）